In [129]:
"""
Generating random trajectories

Game Idea:
    - Battleship
    - With offense & defense
"""

# Base Python
import itertools

# Extended Python
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import matplotlib.colors as mcolors
from plotly.subplots import make_subplots

# Constants
ACC_GRAVITY = -9.81  # (m/s^2)
FULL_HEIGHT = 700  # 900
FULL_WIDTH = 1100  # 1500
STARTING_POSITION = 0
STARTING_TIME = 0

In [130]:
def generate_trajectory(velocity_i, theta, phi, dt):
    """ Generate trajectory based on initial conditions """

    # Calculate velocity vectors
    v_z = velocity_i * np.sin(np.deg2rad(phi))
    v_xy = velocity_i * np.cos(np.deg2rad(phi))
    v_x = v_xy * np.cos(np.deg2rad(theta))
    v_y = v_xy * np.sin(np.deg2rad(theta))

    t = np.array([STARTING_TIME])
    x = np.array([STARTING_POSITION])
    y = np.array([STARTING_POSITION])
    z = np.array([STARTING_POSITION])

    while z[-1] >= STARTING_POSITION:
        t_i = t[-1]
        x_i = x[-1]
        y_i = y[-1]
        z_i = z[-1]

        # Calculate new positions
        x_f = x_i + (v_x * dt)  # ignore friction
        y_f = y_i + (v_y * dt)  # ignore friction
        z_f = z_i + (v_z * dt) + (0.5 * ACC_GRAVITY * dt ** 2)

        # Calculate new velocities (ignore friction, only z-component changes)
        v_z = v_z + (ACC_GRAVITY * dt)

        # Update trajectories
        t = np.append(t, t_i + dt)
        x = np.append(x, x_f)
        y = np.append(y, y_f)
        z = np.append(z, z_f)

    traj = pd.DataFrame({
        "time": t,
        "position_x": x,
        "position_y": y,
        "position_z":z
    })

    return traj

In [131]:
def polar_to_cartesian(r, theta, phi):
     """ Convert polar coordinates to cartesian """
     theta_rad = np.deg2rad(theta)
     phi_rad = np.deg2rad(phi)

     x = r * np.cos(phi_rad) * np.cos(theta_rad)
     y = r * np.cos(phi_rad) * np.sin(theta_rad)
     z = r * np.sin(phi_rad)

     return x, y, z

In [132]:
def make_face_simplices(array_a, array_b, n):
    """
    Create all triagulations of a face between two arrays

    A-------B---o ...
    |\      |
    | \  1  |     
    |  \    |
    |   \   |     ...
    | 2  \  |
    |     \ |
    C-------D---o ...

    (A, B, D) creates triangle 1
    (A, C, D) creates triangle 2
    
    ... repeat for face between full arrays

    """
    simplices = []
    # First Half
    for i in range(n - 1):
        s = np.array([array_a[i], array_a[i + 1], array_b[i]])
        simplices.append(s)

    # Second Half
    for i in range(n - 1):
        s = np.array([array_b[i], array_b[i + 1], array_a[i + 1]])
        simplices.append(s)
    
    return simplices

In [133]:
def generate_polar_simplices(n_points):
	"""
	-- FOR TOP AND BOTTOM FACES --

	near bottom: i * n for i in [0, n - 1]
	far bottom:  i * n for i in [n, 2n - 1]

	near top: i * n - 1 for i in [1, n + 1]
	far top:  i * n - 1 for i in [n + 1, 2n + 1]

	-- FOR LEFT AND RIGHT SIDE FACES --

	near left: [0, n-1]
	far left: [n^2, n^2 + n - 1]

	near right: [n^2 - n, n^2 - 1]
	far right: [2n^2 -n , 2n^2 - 1]

	Then, given any two arrays x and y:

	[ x[i], x[i + 1], y[i + 1] ]  <- first half of triangles
	[ y[i], y[i + 1], x[i]     ]  <- second half of triangles
	"""
	# Bottom and top rows
	near_bottom = [i * n_points for i in range(n_points)]
	far_bottom = [i * n_points for i in range(n_points, 2 * n_points)]
	near_top = [i * n_points - 1 for i in range(1, n_points + 1)]
	far_top = [i * n_points - 1 for i in range(n_points + 1, 2 * n_points + 1)]

	# Left and right rows
	near_right = [i for i in range(n_points)]
	far_right = [i for i in range(n_points ** 2, n_points ** 2 + n_points)]
	near_left = [i for i in range(n_points ** 2 - n_points, n_points ** 2)]
	far_left = [i for i in range(2 * n_points ** 2 - n_points, 2 * n_points ** 2)]

	simplices = []
	simplices.extend(make_face_simplices(near_right, far_right, n_points))    # Right face
	simplices.extend(make_face_simplices(near_left, far_left, n_points))      # Left face
	simplices.extend(make_face_simplices(near_top, far_top, n_points))        # Top face
	simplices.extend(make_face_simplices(near_bottom, far_bottom, n_points))  # Bottom face
	simplices.extend(make_face_simplices(near_bottom, near_top, n_points))    # Top face
	simplices.extend(make_face_simplices(far_bottom, far_top, n_points))      # Bottom face

	# Stack in N x 3 matrix
	simplices = np.vstack(simplices)

	return simplices

In [134]:
def generate_sector(r_min, r_max, theta_min, theta_max, phi_min, phi_max, n_points=10):
    """ 
    Generate sector representing a 3D slice in a polar coordinate frame.
    Convert to cartesian.
    Calculate simplices for triangulation.
    """
    r_mesh = np.array([])
    theta_mesh = np.array([])
    phi_mesh = np.array([])

    r = np.linspace(r_min, r_max, n_points)
    theta = np.linspace(theta_min, theta_max, n_points)
    phi = np.linspace(phi_min, phi_max, n_points)

    # Mesh theta phi with r min
    r_temp, theta_temp, phi_temp = np.meshgrid(np.array([r_min]), theta, phi)
    r_mesh = np.append(r_mesh, r_temp.flatten())
    theta_mesh = np.append(theta_mesh, theta_temp.flatten())
    phi_mesh = np.append(phi_mesh, phi_temp.flatten())

    # Mesh theta phi with r max
    r_temp, theta_temp, phi_temp = np.meshgrid(np.array([r_max]), theta, phi)
    r_mesh = np.append(r_mesh, r_temp.flatten())
    theta_mesh = np.append(theta_mesh, theta_temp.flatten())
    phi_mesh = np.append(phi_mesh, phi_temp.flatten())

    # Create a dataframe of catersian coordinates
    x, y, z = polar_to_cartesian(r_mesh, theta_mesh, phi_mesh)
    sector = pd.DataFrame({
        "x": x,
        "y": y,
        "z": z
    })

    # Generate simplicies
    simplices = generate_polar_simplices(n_points)

    return sector, simplices

### Create Trajectory

In [135]:
vel_set = np.arange(100, 200)
angle_xy_set = np.arange(0, 90)
angle_z_set = np.arange(10, 90)
dt = 0.1

all_trajs = []
# Create 10 random trajectories
for i in range(10):
    vel = np.random.choice(vel_set)
    angle_xy = np.random.choice(angle_xy_set)
    angle_z = np.random.choice(angle_z_set)

    traj = generate_trajectory(vel, angle_xy, angle_z, dt)
    traj["object_id"] = str(i)
    all_trajs.append(traj)

all_trajs = pd.concat(all_trajs)


In [136]:
# Plot Trajectories
fig = px.scatter_3d(
    all_trajs, x="position_x", y="position_y", z="position_z", color="object_id", width=FULL_WIDTH, height=FULL_HEIGHT,
)
fig.update_traces(
    marker={"size":3},
    selector={"mode":'markers'}
)
fig.show()

### Create Sector

In [137]:
n_points = 20
sector, simplices = generate_sector(125, 175, 30, 70, 30, 40, n_points=n_points)
sector = sector.reset_index()

In [138]:
sector_fig = go.Figure(
    data=go.Scatter3d(
        x=sector["x"],
        y=sector["y"],
        z=sector["z"],
        text=sector["index"],
        mode="markers",
        marker=dict(color="green", size=3),
        name="Sector"
    )
)
origin_fig = go.Figure(data=go.Scatter3d(x=[0], y=[0], z=[0], marker=dict(color="blue"), name="Origin"))
fig = go.Figure(data=sector_fig.data + origin_fig.data)
fig.update_layout(title="Searching", width=FULL_WIDTH, height=FULL_HEIGHT)
fig.show()

In [155]:
n_points = 20

sector, simplices = generate_sector(125, 175, 30, 70, 10, 20, n_points=n_points)
sector_fig = ff.create_trisurf(
    x=sector['x'].values, 
    y=sector['y'].values,
    z=sector['z'].values,
    simplices=simplices,
    colormap=[mcolors.to_hex('green')] * 2,
    show_colorbar=False
)
sector_fig['data'][0].update(opacity=0.5)

origin_fig = go.Figure(data=go.Scatter3d(x=[0], y=[0], z=[0], marker=dict(color="blue"), name="Origin"))
fig = go.Figure(data=sector_fig.data + origin_fig.data)
fig.update_layout(title="Searching", width=FULL_WIDTH, height=FULL_HEIGHT)
fig.show()